In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

In [2]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = 'G:/PracaMagisterska/Dane/Mono_Discret_Audio'
excl_folders = ['MT2']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=3

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)

# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

print('dataset size: ', len(dataset))
print('train set size: ', len(split.train_sampler))
print('val set size: ', len(split.val_sampler))
print('test set size: ', len(split.test_sampler))
dataset.fx_to_label

dataset size:  123552
train set size:  88956
val set size:  9885
test set size:  24711


{'808': 0,
 'BD2': 1,
 'BMF': 2,
 'DPL': 3,
 'DS1': 4,
 'FFC': 5,
 'MGS': 6,
 'OD1': 7,
 'RAT': 8,
 'RBM': 9,
 'SD1': 10,
 'TS9': 11,
 'VTB': 12}

In [3]:
# model
input_size = 87  # Update this to match the feature size of your input
hidden_size = 128  # You can adjust this as needed
num_layers = 2  # You can adjust this as needed

multinet = models.MultiRNN(n_classes=dataset.num_fx, n_settings=dataset.max_num_settings, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)
# optimizer
optimizer_multinet = optim.Adam(multinet.parameters(), lr=0.001)
# loss function
# loss function
loss_func_fx = nn.CrossEntropyLoss()
loss_func_set = nn.MSELoss(reduction='mean')

print(multinet)
print('Trainable Params: ', sum(p.numel() for p in multinet.parameters() if p.requires_grad))

MultiRNN(
  (lstm): LSTM(87, 128, num_layers=2, batch_first=True)
  (fc1_a): Linear(in_features=128, out_features=120, bias=True)
  (fc2_a): Linear(in_features=120, out_features=60, bias=True)
  (out_a): Linear(in_features=60, out_features=13, bias=True)
  (fc1_b): Linear(in_features=128, out_features=120, bias=True)
  (fc2_b): Linear(in_features=120, out_features=60, bias=True)
  (out_b): Linear(in_features=60, out_features=3, bias=True)
)
Trainable Params:  289656


In [4]:
# SAVE
models_folder = 'G:/PracaMagisterska/Kod_na_magisterke/gfx-classifier_RNN_version-models_and_results/models'
model_name = '20201110_multinet_mono_disc_best'
results_folder = 'G:/PracaMagisterska/Kod_na_magisterke/gfx-classifier_RNN_version-models_and_results/result'
results_subfolder = '20201110_multinet_mono_disc'

In [5]:
# TRAIN and TEST MultiNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_multi_net(
        model=multinet,
        optimizer=optimizer_multinet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch, 
        loss_function_fx=loss_func_fx, 
        loss_function_set=loss_func_set,
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_multi_net(
        model=multinet,
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        loss_function_fx=loss_func_fx, 
        loss_function_set=loss_func_set,
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_multi_net(
        model=multinet,
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        loss_function_fx=loss_func_fx, 
        loss_function_set=loss_func_set,
        device=device
    )
    # save model
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(multinet, '%s/%s' % (models_folder, model_name))
        print('\n=== saved best model ===\n')
        
    # append results
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[5000/88956 (6%)]	Total Loss: 138.1869	Avg Loss: 0.0276
Train Epoch: 0	[10000/88956 (11%)]	Total Loss: 266.4475	Avg Loss: 0.0266
Train Epoch: 0	[15000/88956 (17%)]	Total Loss: 392.5159	Avg Loss: 0.0262
Train Epoch: 0	[20000/88956 (22%)]	Total Loss: 509.9446	Avg Loss: 0.0255
Train Epoch: 0	[25000/88956 (28%)]	Total Loss: 624.0031	Avg Loss: 0.0250
Train Epoch: 0	[30000/88956 (34%)]	Total Loss: 735.3319	Avg Loss: 0.0245
Train Epoch: 0	[35000/88956 (39%)]	Total Loss: 844.0400	Avg Loss: 0.0241
Train Epoch: 0	[40000/88956 (45%)]	Total Loss: 951.9737	Avg Loss: 0.0238
Train Epoch: 0	[45000/88956 (51%)]	Total Loss: 1059.1489	Avg Loss: 0.0235
Train Epoch: 0	[50000/88956 (56%)]	Total Loss: 1164.3401	Avg Loss: 0.0233
Train Epoch: 0	[55000/88956 (62%)]	Total Loss: 1265.7286	Avg Loss: 0.0230
Train Epoch: 0	[60000/88956 (67%)]	Total Loss: 1362.9278	Avg Loss: 0.0227
Train Epoch: 0	[65000/88956 (73%)]	Total Loss: 1457.1253	Avg Loss: 0.0224
Train Epoch: 0	[70000/88956 (79%)]	Total Loss: 1

In [6]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  42.02077431539188
Epoch:  47

Accuracy:  42.36722306525038
Epoch:  48

Accuracy:  42.70972441422848
Epoch:  48



In [8]:
# SAVE RESULTS - all losses, all correct, best results
all_train_losses_npy = np.array(all_train_losses, dtype=object)
all_train_correct_npy = np.array(all_train_correct, dtype=object)
best_train_results_npy = np.array(all_train_results[43], dtype=object)

all_val_losses_npy = np.array(all_val_losses, dtype=object)
all_val_correct_npy = np.array(all_val_correct, dtype=object)
best_val_results_npy = np.array(all_val_results[33], dtype=object)

all_test_losses_npy = np.array(all_test_losses, dtype=object)
all_test_correct_npy = np.array(all_test_correct, dtype=object)
best_test_results_npy = np.array(all_test_results[39], dtype=object)

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()), dtype=object)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_train_results')), arr=best_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_val_results')), arr=best_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_test_results')), arr=best_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'fx_labels')), arr=fx_labels_npy)